# Day 22
## Part 1

In [1]:
from collections import deque
from itertools import count, islice
from pyrsistent import pvector, pdeque


def parse_data(s):
    decks = s.split('\n\n')
    result = pvector()
    for deck in decks:
        result = result.append(pdeque(int(x) for x in deck.strip().splitlines()[1:]))
    return result

In [2]:
def game_round(decks):
    if decks[0][0] > decks[1][0]:
        winner = 0
        loser = 1
    else:
        winner = 1
        loser = 0
        
    decks = (
        decks.set(winner, (
            decks[winner]
            .append(decks[winner][0])
            .append(decks[loser][0])
            .popleft()))
        .set(loser, decks[loser].popleft())) 
    
    return decks


def game(decks):
    while all(decks):
        decks = game_round(decks)
        
    return decks


def score(decks):
    if decks[0]:
        winner = 0
    else:
        winner = 1
        
    return sum(x * y for x, y in zip(count(1), reversed(decks[winner])))


def part_1(decks):
    return score(game(decks))

In [3]:
test_decks = parse_data('''Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
''')

assert part_1(test_decks) == 306

In [4]:
decks = parse_data(open('input').read())
part_1(decks)

31269

## Part 2

In [5]:
def recursive_combat(decks):
    memory = set()
    
    while all(decks):
        if decks in memory:
            return (decks, 0)
        else:
            memory.add(decks)

        if decks[0][0] < len(decks[0]) and decks[1][0] < len(decks[1]):
            _, winner = recursive_combat(
                pvector([
                    pdeque(islice(decks[0], 1, decks[0][0] + 1)),
                    pdeque(islice(decks[1], 1, decks[1][0] + 1))
                ])
            )
            loser = 1 if winner == 0 else 0
        elif decks[0][0] > decks[1][0]:
            winner = 0
            loser = 1
        else:
            winner = 1
            loser = 0
            
        decks = (
            decks.set(winner, (
                decks[winner]
                .append(decks[winner][0])
                .append(decks[loser][0])
                .popleft()))
            .set(loser, decks[loser].popleft())) 


    if decks[0]:
        return (decks, 0)
    else:
        return (decks, 1)
    
    
def part_2(decks):
    decks, winner = recursive_combat(decks)
    return sum(x * y for x, y in zip(count(1), reversed(decks[winner])))



In [6]:
assert part_2(test_decks) == 291

In [7]:
looping_decks = parse_data('''Player 1:
43
19

Player 2:
2
29
14''')

part_2(looping_decks)

105

In [8]:
%time part_2(decks)

CPU times: user 1min 36s, sys: 13.2 ms, total: 1min 36s
Wall time: 1min 36s


31151

I switched to using `pyrsistent` for part 2 so I could check the memory, though I'm not sure that was a good idea, I seem to recall it performing poorly for other card based problems. I'm not changing it now, though.